# Homework 2

# 1

## (1)

$$
\begin{aligned}
b_1 &= argmin_{\beta_1} || Y - X_1 \beta_1||^2 \\
\end{aligned}
$$
考虑其梯度：

$$
\begin{aligned}
\frac{\partial}{\partial \beta_1} || Y - X_1 \beta_1||^2 &= -2 X_1^T (Y - X_1 \beta_1) \\
\end{aligned}
$$

故当梯度为0时，有：
$$
\begin{aligned}
b_1 = \beta_1 &= (X_1^T X_1)^{-1} X_1^T Y \\
\end{aligned}
$$

## (2)

$$
\begin{aligned}
(c_1, c_2) = argmin_{\gamma_1, \gamma_2} || Y - X_1 \gamma_1 - X_2 \gamma_2||^2 \\
\end{aligned}
$$
考虑其梯度：

$$
\begin{cases}
\frac{\partial}{\partial \gamma_1} || Y - X_1 \gamma_1 - X_2 \gamma_2||^2 &= -2 X_1^T (Y - X_1 \gamma_1 - X_2 \gamma_2) \\
\frac{\partial}{\partial \gamma_2} || Y - X_1 \gamma_1 - X_2 \gamma_2||^2 &= -2 X_2^T (Y - X_1 \gamma_1 - X_2 \gamma_2) \\
\end{cases}
$$

故当梯度为0时，有：
$$
\begin{cases}
\gamma_1 = (X_1^T X_1)^{-1} X_1^T (Y - X_2 \gamma_2) \\
\gamma_2 = (X_2^T X_2)^{-1} X_2^T (Y - X_1 \gamma_1) = (X_2^T X_2)^{-1} X_2^T (Y - X_1 (X_1^T X_1)^{-1} X_1^T (Y - X_2 \gamma_2)) \\
\end{cases}
$$

可以解得：
$$
\gamma_2 = (X_2^T X_2)^{-1} X_2^T M_1 Y - (X_2^T X_2)^{-1} X_2^T P_1 X_2 \gamma_2,
$$
其中：
$$
\begin{cases}
M_1 = I - X_1 (X_1^T X_1)^{-1} X_1^T \\
P_1 = X_1 (X_1^T X_1)^{-1} X_1^T \\
\end{cases}
$$

故：
$$
\gamma_2 = (X_2^T M_1 X_2)^{-1} X_2^T M_1 Y
$$

进而可知：
$$
\gamma_1 = (X_1^T M_2 X_1)^{-1} X_1^T M_2 Y
$$

综上：
$$
\begin{cases}
c_1 = \gamma_1 = (X_1^T M_2 X_1)^{-1} X_1^T M_2 Y \\
c_2 = \gamma_2 = (X_2^T M_1 X_2)^{-1} X_2^T M_1 Y \\
\end{cases}
$$

# 2

由1知：

$$
\begin{aligned}
\hat \beta &= (X^T X)^{-1} X^T Y \\
&= (X^T X)^{-1} X^T (X \beta + \epsilon) \\
&= \beta + (X^T X)^{-1} X^T \epsilon \\
\end{aligned}
$$

由于$\epsilon$的条件期望为0，故$\hat \beta$的条件期望为$\beta$，故$\hat \beta$是（条件）无偏估计。

下面证明其是BLUE:
为方便起见，我们记$\hat \beta = (X^T X)^{-1}X^T Y = A Y$，其中$A = (X^T X)^{-1} X^T$。

如果有另一个线性无偏估计$\tilde \beta := B Y$，其中$B$是一个线性映射，且$E(\tilde \beta | X) = \beta$。

注意到：
$$
\mathbb E (A Y | X) = \mathbb E (B Y | X) = \beta
$$

即：
$$
AX\beta = BX\beta, \forall \beta \\
$$

故：
$$
(B - A)X = 0 \\
(B - A)A^T = 0
$$

进一步注意到：

故：
$$
\begin{aligned}
Var(\tilde \beta | X) &= Var(BY | X) \\
&= Var((B-A)Y + AY | X) \\
&= Var((B-A)\epsilon | X) + Var(A \epsilon | X) + 2 Cov((B-A)\epsilon, A \epsilon | X) \\
&= Var((B-A)\epsilon | X) + Var(A \epsilon | X) + 2 \sigma^2 (B-A)A^T \\
&= Var((B-A)\epsilon | X) + Var(A \epsilon | X) \\
&\ge Var(A \epsilon | X) \\
&= Var(\hat \beta | X) \\
\end{aligned}
$$

故$\hat \beta$是BLUE。

# 3

In [58]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

t = np.arange(1, 51)
X1 = np.array([np.ones_like(t), t]).copy().T
X2 = np.array([t**2]).copy().T

def func(a, b, c, mu, sigma):
    return lambda x: a + b * x + c * x**2 + np.random.normal(mu, sigma, x.size)

f = func(10, 1, 2, 0, 0.01)
Y = f(t)

In [70]:
X1_tilde = X1.copy()
X1_tilde_1 = X1[:, 0] / np.sqrt(X1[:, 0] @ X1[:, 0])
u_2 = X1[:, 1] - X1_tilde_1 @ X1[:, 1] * X1_tilde_1
X1_tilde_2 = u_2 / np.sqrt(u_2 @ u_2)
X1_tilde = np.concatenate([X1_tilde_1.reshape(-1, 1), X1_tilde_2.reshape(-1, 1)], axis=1)

X2_tilde = (X2[:, 0] - X2[:, 0] @ X1_tilde[:, 0] * X1_tilde[:, 0] - X2[:, 0] @ X1_tilde[:, 1] * X1_tilde[:, 1])[:, np.newaxis]

In [71]:
model_1 = LinearRegression(fit_intercept=False)
model_1.fit(X1_tilde, Y)

print('b1 = ', model_1.coef_[1])

b1 =  10510.299565774136


In [72]:
model_2 = LinearRegression(fit_intercept=False)
model_2.fit(np.concatenate([X1_tilde, X2_tilde], axis=1), Y)

print('gamma_1 = ', model_2.coef_[1])

gamma_1 =  10510.29956577413
